In [1]:
import pandas as pd
data_bin = pd.read_csv(r'C:\Users\erich\Desktop\DS_project\data\data_30_sentences.csv')

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

lemmatizer = WordNetLemmatizer()

def clean_bin_teen(df):
    df = df[df['HTML_Content'].isin(['Explicit', 'General Audiences', 'Mature', 'Not Rated', 'Teen And Up Audiences'])]

    df = df.dropna(subset=['TXT_Content'])

    def clean_text(text):

        if not isinstance(text, str):
            text = str(text)

        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        tokenized_text = word_tokenize(text)
        cleaned_text = [lemmatizer.lemmatize(word) for word in tokenized_text if word not in set(stopwords.words('english'))]

        return ' '.join(cleaned_text)

    df['TXT_Content'] = df['TXT_Content'].apply(clean_text)
    
    df['HTML_Content'] = df['HTML_Content'].apply(lambda x: 'Others' if x != 'Teen And Up Audiences' else x)

    return df

data_bin = clean_bin_teen(data_bin)
print(data_bin)

C:\Users\erich\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


               HTML_Content                                        TXT_Content
0     Teen And Up Audiences  huckleberry finn stood front old wooden door m...
1     Teen And Up Audiences  right huck trying figure every sort way could ...
2                    Others  three buried mother christmas eve thing huck c...
3     Teen And Up Audiences  air hot brow wet mind exhausted st petersburg ...
4                    Others  episode genre al chaos chaos space time vortex...
...                     ...                                                ...
6036  Teen And Up Audiences  seen news pat asked robin chance hang coat mor...
6037  Teen And Up Audiences  free next friday night murphy smiled phone rob...
6038                 Others  turned walked away wan na say come baby give w...
6039                 Others  perfect word tended apply liberally one often ...
6040                 Others  heaven fact unlike earth pro con pro include t...

[6005 rows x 2 columns]


In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, f1_score

tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data_bin['TXT_Content'])
y = data_bin['HTML_Content']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = MultinomialNB(alpha=0.5, fit_prior=True)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred, pos_label='Teen And Up Audiences'):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred, pos_label='Teen And Up Audiences'):.4f}")


Accuracy: 0.7019
Recall: 0.0138
F1-score: 0.0272
